### Install Libraries

In [14]:
import matplotlib as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import shutil
import random
from shutil import copyfile
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Data of Cats and Dogs

In [ ]:
local_zip = './data/archive.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('./data')
zip_ref.close()

In [6]:
base_directory = './data/Animal Images/'
source_dog = os.path.join(base_directory, 'dogs')
source_cat = os.path.join(base_directory, 'cats')

print(f"There are {len(os.listdir(source_cat))} Cat Images")
print(f"There are {len(os.listdir(source_dog))} Dog Images")

There are 15060 Cat Images
There are 15002 Dog Images


### Create a new directory for training and validation data 

In [8]:

os.makedirs('./cat-dogs/train/dogs')
os.makedirs('./cat-dogs/train/cats')
os.makedirs('./cat-dogs/validation/cats')
os.makedirs('./cat-dogs/validation/dogs')

In [9]:

def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):

  lista = os.listdir(SOURCE_DIR)
  lista_random = random.sample(lista,len(lista)) # input --> Type list

  percentage = int(SPLIT_SIZE*len(lista_random)) # Puede darse el caso que no sea natural

  data = [lista_random[i:i+percentage] for i in range(0,len(lista_random),percentage)] #forma más rápida pero falta el if
  training_data = data[0]; # ['890.jpg', '12409.jpg', '9285.jpg', '8606.jpg']
  test_data = data[1];


  for i in training_data:
    if (  os.path.getsize(SOURCE_DIR + i) != 0 ): # /tmp/PetImages/Cat/7364.jpg
      shutil.copy(SOURCE_DIR + i, (TRAINING_DIR))
    else:
      print(i + " is zero length, so ignoring.")
  for i in test_data:
    if (os.path.getsize(SOURCE_DIR + i) !=  0):
      shutil.copy(SOURCE_DIR + i, (VALIDATION_DIR)) # copyfile --> da error
    else:
      print(i + " is zero length, so ignoring.")
      


In [11]:
CAT_SOURCE_DIR = './data/Animal Images/cats/'
DOG_SOURCE_DIR = './data/Animal Images/dogs/'

CAT_TRAIN_DIR = './cat-dogs/train/cats/' 
DOG_TRAIN_DIR = './cat-dogs/train/dogs/'

CAT_VAL_DIR = './cat-dogs/validation/cats/'
DOG_VAL_DIR = './cat-dogs/validation/dogs/'

SPLIT_SIZE = 0.9

split_data(CAT_SOURCE_DIR, CAT_TRAIN_DIR, CAT_VAL_DIR, SPLIT_SIZE)
split_data(DOG_SOURCE_DIR, DOG_TRAIN_DIR, DOG_VAL_DIR, SPLIT_SIZE)

In [12]:
print(f"There are {len(os.listdir(CAT_TRAIN_DIR))} in training Cat Images")
print(f"There are {len(os.listdir(DOG_TRAIN_DIR))} in training Dog Images")
print(f"There are {len(os.listdir(CAT_VAL_DIR))} in validation Cat Images")
print(f"There are {len(os.listdir(DOG_VAL_DIR))} in validation Dog Images")

There are 13554 in training Cat Images
There are 13501 in training Dog Images
There are 1506 in validation Cat Images
There are 1501 in validation Dog Images


### Pre-processing Image

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
      
  train_data_gen = ImageDataGenerator(rescale = 1./255,
                                      rotation_range = 45,
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True,
                                      width_shift_range = 0.2,
                                      height_shift_range = 0.2,
                                      fill_mode = "nearest")

 
  train_generator = train_data_gen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=20,
                                                      class_mode="binary",
                                                      target_size=(150, 150))

 
  validation_data_gen = ImageDataGenerator( rescale = 1.0/255. )


  validation_generator = validation_data_gen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=20,
                                                                class_mode="binary",
                                                                target_size=(150, 150))
  
  return train_data_gen, validation_generator


### Create CNN and Fit model

In [ ]:
def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu", input_shape = (150,150,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3),  activation ='relu'),
    tf.keras.layers.MaxPooling2D(2,2), # Agrupacion máx de 2x2
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Dropout(0.2), # Agregamos la probabilidad de que se apaguen aleatoriamente para que los pesos no sean disparejos

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'), # 512 neuronas

    tf.keras.layers.Dense(1, activation='sigmoid')  # 1 output
  ])

  model.compile(optimizer = "rmsprop",
              loss = "categorical_crossentropy", # cambia cuando solo son 2 clases
              metrics = ['accuracy'])
  
  return model

model = create_model()
model.summary()

In [ ]:
# Train the model
history = model.fit(TRAIN_GEN,
                    epochs = 12,
                    verbose = 1,
                    validation_data = VAL_GEN)

### Prediction

In [ ]:
import numpy as np
import keras.utils as image

archivo = files.upload()
index = 0

mayor = 0.
for x in archivo.keys():
  path = "/content/"+ x
  img = image.load_img(path, target_size = (90,90))
  
  y = image.img_to_array(img)
  y /= 255
  y = np.expand_dims(y, axis=0)
  images = np.vstack([y])

  classes = model.predict(images, batch_size = 10)
  print(classes[0])
  for count,value in enumerate(classes[0]):
    if value > mayor:
      mayor = value
      index = count
  print(x + "is a " + str(index))

### Graphics Accuracy

In [ ]:

acc,val_acc,loss,val_loss = history.history["accuracy"],history.history["val_accuracy"],history.history["loss"],history.history["val_loss"]
epochs = range(len(acc))

plt.figure()
plt.plot(epochs,acc, "r", "Training Accuracy")
plt.plot(epochs,val_acc,"b","Validation Accuracy")
plt.title('Training and validation accuracy')
plt.grid()
plt.show()

print("-------------------------------------------")


plt.figure()
plt.plot(epochs,loss,"r", "Training Loss")
plt.plot(epochs,val_loss,"b", "Validation Loss")
plt.grid()
plt.show()